In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter #RecursiveCharacterTextSplitter

# Initialize the text splitter
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# Initialize the document loader
loader = UnstructuredFileLoader("./files/BAU_Report_May.docx")

# Load the document and split it into chunks
loader.load_and_split(text_splitter=splitter)




Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600


14